<a href="https://colab.research.google.com/github/kipsangmarion/webscraper/blob/main/WebScraper.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Dependencies:

In [2]:
from bs4 import BeautifulSoup
import lxml
import requests
import pandas as pd
import numpy as np

In [3]:
# Replace with the URL of the website you want to scrape
urls = ['https://www.thegef.org/projects-operations/database?f%5B0%5D=focal_areas%3A2207&search=&page=0',
        'https://www.thegef.org/projects-operations/database?f%5B0%5D=focal_areas%3A2207&search=&page=1',
        'https://www.thegef.org/projects-operations/database?f%5B0%5D=focal_areas%3A2207&search=&page=2',
        'https://www.thegef.org/projects-operations/database?f%5B0%5D=focal_areas%3A2207&search=&page=3',
        'https://www.thegef.org/projects-operations/database?f%5B0%5D=focal_areas%3A2207&search=&page=4',
        'https://www.thegef.org/projects-operations/database?f%5B0%5D=focal_areas%3A2207&search=&page=5',
        'https://www.thegef.org/projects-operations/database?f%5B0%5D=focal_areas%3A2207&search=&page=6',
        'https://www.thegef.org/projects-operations/database?f%5B0%5D=focal_areas%3A2207&search=&page=7',
        'https://www.thegef.org/projects-operations/database?f%5B0%5D=focal_areas%3A2207&search=&page=8',
        'https://www.thegef.org/projects-operations/database?f%5B0%5D=focal_areas%3A2207&search=&page=9',
        'https://www.thegef.org/projects-operations/database?f%5B0%5D=focal_areas%3A2207&search=&page=10',
        'https://www.thegef.org/projects-operations/database?f%5B0%5D=focal_areas%3A2207&search=&page=11',
        'https://www.thegef.org/projects-operations/database?f%5B0%5D=focal_areas%3A2207&search=&page=12',
        'https://www.thegef.org/projects-operations/database?f%5B0%5D=focal_areas%3A2207&search=&page=13',
        'https://www.thegef.org/projects-operations/database?f%5B0%5D=focal_areas%3A2207&search=&page=14',
        'https://www.thegef.org/projects-operations/database?f%5B0%5D=focal_areas%3A2207&search=&page=15',
        'https://www.thegef.org/projects-operations/database?f%5B0%5D=focal_areas%3A2207&search=&page=16',
        'https://www.thegef.org/projects-operations/database?f%5B0%5D=focal_areas%3A2207&search=&page=17',
        'https://www.thegef.org/projects-operations/database?f%5B0%5D=focal_areas%3A2207&search=&page=18',
        'https://www.thegef.org/projects-operations/database?f%5B0%5D=focal_areas%3A2207&search=&page=19',
        'https://www.thegef.org/projects-operations/database?f%5B0%5D=focal_areas%3A2207&search=&page=20',
        'https://www.thegef.org/projects-operations/database?f%5B0%5D=focal_areas%3A2207&search=&page=21',
        'https://www.thegef.org/projects-operations/database?f%5B0%5D=focal_areas%3A2207&search=&page=22',
        'https://www.thegef.org/projects-operations/database?f%5B0%5D=focal_areas%3A2207&search=&page=23',
        'https://www.thegef.org/projects-operations/database?f%5B0%5D=focal_areas%3A2207&search=&page=24',
        'https://www.thegef.org/projects-operations/database?f%5B0%5D=focal_areas%3A2207&search=&page=25',
        'https://www.thegef.org/projects-operations/database?f%5B0%5D=focal_areas%3A2207&search=&page=26',
        'https://www.thegef.org/projects-operations/database?f%5B0%5D=focal_areas%3A2207&search=&page=27',
        'https://www.thegef.org/projects-operations/database?f%5B0%5D=focal_areas%3A2207&search=&page=28',
        'https://www.thegef.org/projects-operations/database?f%5B0%5D=focal_areas%3A2207&search=&page=29',
        'https://www.thegef.org/projects-operations/database?f%5B0%5D=focal_areas%3A2207&search=&page=30',
        'https://www.thegef.org/projects-operations/database?f%5B0%5D=focal_areas%3A2207&search=&page=31',
        'https://www.thegef.org/projects-operations/database?f%5B0%5D=focal_areas%3A2207&search=&page=32',
        'https://www.thegef.org/projects-operations/database?f%5B0%5D=focal_areas%3A2207&search=&page=33',
        'https://www.thegef.org/projects-operations/database?f%5B0%5D=focal_areas%3A2207&search=&page=34',
        'https://www.thegef.org/projects-operations/database?f%5B0%5D=focal_areas%3A2207&search=&page=35',
        'https://www.thegef.org/projects-operations/database?f%5B0%5D=focal_areas%3A2207&search=&page=36',
        'https://www.thegef.org/projects-operations/database?f%5B0%5D=focal_areas%3A2207&search=&page=37',
        'https://www.thegef.org/projects-operations/database?f%5B0%5D=focal_areas%3A2207&search=&page=38',
        'https://www.thegef.org/projects-operations/database?f%5B0%5D=focal_areas%3A2207&search=&page=39',
        'https://www.thegef.org/projects-operations/database?f%5B0%5D=focal_areas%3A2207&search=&page=40',
        'https://www.thegef.org/projects-operations/database?f%5B0%5D=focal_areas%3A2207&search=&page=41',
        'https://www.thegef.org/projects-operations/database?f%5B0%5D=focal_areas%3A2207&search=&page=42',
        'https://www.thegef.org/projects-operations/database?f%5B0%5D=focal_areas%3A2207&search=&page=43',
        'https://www.thegef.org/projects-operations/database?f%5B0%5D=focal_areas%3A2207&search=&page=44',
        'https://www.thegef.org/projects-operations/database?f%5B0%5D=focal_areas%3A2207&search=&page=54',
        'https://www.thegef.org/projects-operations/database?f%5B0%5D=focal_areas%3A2207&search=&page=46',
        'https://www.thegef.org/projects-operations/database?f%5B0%5D=focal_areas%3A2207&search=&page=47',
        'https://www.thegef.org/projects-operations/database?f%5B0%5D=focal_areas%3A2207&search=&page=48',
        'https://www.thegef.org/projects-operations/database?f%5B0%5D=focal_areas%3A2207&search=&page=49',
        'https://www.thegef.org/projects-operations/database?f%5B0%5D=focal_areas%3A2207&search=&page=50',
        'https://www.thegef.org/projects-operations/database?f%5B0%5D=focal_areas%3A2207&search=&page=51',
        'https://www.thegef.org/projects-operations/database?f%5B0%5D=focal_areas%3A2207&search=&page=52',
        'https://www.thegef.org/projects-operations/database?f%5B0%5D=focal_areas%3A2207&search=&page=53',
        'https://www.thegef.org/projects-operations/database?f%5B0%5D=focal_areas%3A2207&search=&page=54',
        'https://www.thegef.org/projects-operations/database?f%5B0%5D=focal_areas%3A2207&search=&page=55',
        'https://www.thegef.org/projects-operations/database?f%5B0%5D=focal_areas%3A2207&search=&page=56',
        'https://www.thegef.org/projects-operations/database?f%5B0%5D=focal_areas%3A2207&search=&page=57',
        'https://www.thegef.org/projects-operations/database?f%5B0%5D=focal_areas%3A2207&search=&page=58',
        'https://www.thegef.org/projects-operations/database?f%5B0%5D=focal_areas%3A2207&search=&page=59',
        'https://www.thegef.org/projects-operations/database?f%5B0%5D=focal_areas%3A2207&search=&page=60',
        'https://www.thegef.org/projects-operations/database?f%5B0%5D=focal_areas%3A2207&search=&page=61',
        'https://www.thegef.org/projects-operations/database?f%5B0%5D=focal_areas%3A2207&search=&page=62',
        'https://www.thegef.org/projects-operations/database?f%5B0%5D=focal_areas%3A2207&search=&page=63',
        'https://www.thegef.org/projects-operations/database?f%5B0%5D=focal_areas%3A2207&search=&page=64',
        'https://www.thegef.org/projects-operations/database?f%5B0%5D=focal_areas%3A2207&search=&page=65',
        'https://www.thegef.org/projects-operations/database?f%5B0%5D=focal_areas%3A2207&search=&page=66',
        'https://www.thegef.org/projects-operations/database?f%5B0%5D=focal_areas%3A2207&search=&page=67',
        'https://www.thegef.org/projects-operations/database?f%5B0%5D=focal_areas%3A2207&search=&page=68',
        'https://www.thegef.org/projects-operations/database?f%5B0%5D=focal_areas%3A2207&search=&page=69',
        'https://www.thegef.org/projects-operations/database?f%5B0%5D=focal_areas%3A2207&search=&page=70',
        'https://www.thegef.org/projects-operations/database?f%5B0%5D=focal_areas%3A2207&search=&page=71',
        'https://www.thegef.org/projects-operations/database?f%5B0%5D=focal_areas%3A2207&search=&page=72',
        'https://www.thegef.org/projects-operations/database?f%5B0%5D=focal_areas%3A2207&search=&page=73',
        'https://www.thegef.org/projects-operations/database?f%5B0%5D=focal_areas%3A2207&search=&page=74',
        'https://www.thegef.org/projects-operations/database?f%5B0%5D=focal_areas%3A2207&search=&page=75',
        'https://www.thegef.org/projects-operations/database?f%5B0%5D=focal_areas%3A2207&search=&page=76'
        ]

# Send an HTTP request to the URL
project_links = []
for url in urls:
  response = requests.get(url)

  soup = BeautifulSoup(response.text, 'html.parser')

  # Find the table that contains the links (you may need to inspect the HTML)
  table = soup.find('table')

  # Loop through rows in the table
  for row in table.find_all('tr'):
    # Check if there are any <td> elements in this row
    td_elements = row.find_all('td')
    if td_elements:
      # Assuming the links are in the first column (index 0)
      link = td_elements[0].find('a')
      if link:
        link_text = link.get('href')
        full_link = "https://www.thegef.org" + link_text
        project_links.append(full_link)

project_links

['https://www.thegef.org/projects-operations/projects/11318',
 'https://www.thegef.org/projects-operations/projects/11315',
 'https://www.thegef.org/projects-operations/projects/11312',
 'https://www.thegef.org/projects-operations/projects/11310',
 'https://www.thegef.org/projects-operations/projects/11309',
 'https://www.thegef.org/projects-operations/projects/11306',
 'https://www.thegef.org/projects-operations/projects/11305',
 'https://www.thegef.org/projects-operations/projects/11303',
 'https://www.thegef.org/projects-operations/projects/11302',
 'https://www.thegef.org/projects-operations/projects/11285',
 'https://www.thegef.org/projects-operations/projects/11284',
 'https://www.thegef.org/projects-operations/projects/11280',
 'https://www.thegef.org/projects-operations/projects/11279',
 'https://www.thegef.org/projects-operations/projects/11278',
 'https://www.thegef.org/projects-operations/projects/11277',
 'https://www.thegef.org/projects-operations/projects/11276',
 'https:

In [4]:
for project_link in project_links:
  response = requests.get(project_link)

  soup = BeautifulSoup(response.text, 'html.parser')

  # Project title
  title_element = soup.find('span', class_='field--name-title')

  # Extract the text content of the title
  title_text = title_element.get_text(strip=True)

  # Create a list and add the title to it
  project_titles = [title_text]


KeyboardInterrupt: ignored